In [1]:
import tensorflow as tf
import numpy as np

In [2]:

# Load CSV using Pandas
import pandas
filename = './data/training_input.csv'
names = ['id','entryUrl','productLink','odds','winners','startTime','endTime','price','hard_endTime','hasVideo']
df = pandas.read_csv(filename, names=names)
print(df.shape)

(80056, 10)


C:\Users\dan.KWINS\AppData\Local\Continuum\anaconda3\envs\ML\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,4,5,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df.drop(['id', 'entryUrl', 'productLink'], axis=1)
df = df.drop([0], axis=0)


In [4]:
import pandas as pd
df.winners = df.winners.astype(int)
df.startTime = df.startTime.astype(int)
df.endTime = df.endTime.astype(int)
df.price = df.price.str.replace('$','').astype(float)
df.dtypes
df.shape

(80055, 7)

In [5]:
df = df[df.odds.str.contains('varies') != True]
print(df.shape)
df = df.dropna()


(69780, 7)


In [6]:
df.shape


(62445, 7)

In [7]:
df.hard_endTime = pd.to_numeric(df.hard_endTime)
df.hard_endTime = df.hard_endTime.astype(int)


In [8]:
df.hasVideo = pd.to_numeric(df.hasVideo)
df.hasVideo = df.hasVideo.astype(str)
df.odds = pd.to_numeric(df.odds)
df.odds = df.odds.astype(int)

In [9]:
df.dtypes

odds              int32
winners           int32
startTime         int32
endTime           int32
price           float64
hard_endTime      int32
hasVideo         object
dtype: object

In [10]:
df['startTimeDT'] = pd.to_datetime(df['startTime'],unit='s')
df['endTimeDT'] = pd.to_datetime(df['hard_endTime'],unit='s')
df['startTime'] = df['startTime'] - (7*60*60)
df['duration'] = df.endTime - df.startTime

In [11]:
df.describe()

,odds,winners,startTime,endTime,price,hard_endTime,duration
count,62445.000000,62445.000000,6.244500e+04,6.244500e+04,62445.00000,6.244500e+04,6.244500e+04
mean,1585.248843,3.953111,1.528539e+09,1.528853e+09,21.31873,1.529190e+09,3.142785e+05
std,1650.905471,6.517710,3.298784e+06,3.285758e+06,21.41149,3.338298e+06,2.911422e+05
min,1.000000,1.000000,1.517436e+09,1.517558e+09,0.00000,1.517558e+09,-1.925100e+06
25%,300.000000,1.000000,1.525668e+09,1.526045e+09,9.99000,1.526368e+09,5.508000e+04
50%,1300.000000,2.000000,1.528864e+09,1.529219e+09,16.61000,1.529564e+09,1.440000e+05
75%,2000.000000,3.000000,1.531566e+09,1.531757e+09,25.99000,1.532243e+09,6.612600e+05
max,63000.000000,50.000000,1.534329e+09,1.534316e+09,649.00000,1.534316e+09,2.712960e+06


In [12]:
df = df[df['duration'] > 0]
df['startDay'] = df['startTimeDT'].dt.dayofweek.astype(str)
df['startDayOfMonth'] = df['startTimeDT'].dt.day.astype(str)
df['startMonth'] = df['startTimeDT'].map(lambda x: x.month).astype(str)
df['startHour'] = df['startTimeDT'].map(lambda x: x.hour).astype(str)
df['endDay'] = df['endTimeDT'].dt.dayofweek.astype(str)
df['endMonth'] = df['endTimeDT'].map(lambda x: x.month).astype(str)
df['endHour'] = df['endTimeDT'].map(lambda x: x.hour).astype(str)
df['endDayOfMonth'] = df['endTimeDT'].dt.day.astype(str)


In [13]:
x = df[['odds','winners','price','hasVideo',
        'startDay','startHour','startDayOfMonth','endDay',
        'endHour','endDayOfMonth']] 
y = df['duration']
df.head()

,odds,winners,startTime,endTime,price,hard_endTime,hasVideo,startTimeDT,endTimeDT,duration,startDay,startDayOfMonth,startMonth,startHour,endDay,endMonth,endHour,endDayOfMonth
5,300,2,1533633240,1534316340,4.99,1534316340,0,2018-08-07 16:14:00,2018-08-15 06:59:00,683100,1,7,8,16,2,8,6,15
6,2200,1,1533632940,1533704820,32.99,1534316340,0,2018-08-07 16:09:00,2018-08-15 06:59:00,71880,1,7,8,16,2,8,6,15
7,1900,1,1533632760,1534316340,150.00,1534316340,0,2018-08-07 16:06:00,2018-08-15 06:59:00,683580,1,7,8,16,2,8,6,15
8,800,5,1533632760,1534316340,29.22,1534316340,1,2018-08-07 16:06:00,2018-08-15 06:59:00,683580,1,7,8,16,2,8,6,15
9,2500,1,1533632700,1533684900,39.99,1534316340,0,2018-08-07 16:05:00,2018-08-15 06:59:00,52200,1,7,8,16,2,8,6,15


In [14]:
x.columns

Index(['odds', 'winners', 'price', 'hasVideo', 'startDay', 'startHour',
       'startDayOfMonth', 'endDay', 'endHour', 'endDayOfMonth'],
      dtype='object')

In [15]:
cols_to_norm = ['odds', 'winners', 'price']
x[cols_to_norm] = x[cols_to_norm].apply(lambda x: (x - x.min())/(x.max() - x.min()))
x.head()

C:\Users\dan.KWINS\AppData\Local\Continuum\anaconda3\envs\ML\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,odds,winners,price,hasVideo,startDay,startHour,startDayOfMonth,endDay,endHour,endDayOfMonth
5,0.004746,0.020408,0.007689,0,1,16,7,2,6,15
6,0.034905,0.000000,0.050832,0,1,16,7,2,6,15
7,0.030143,0.000000,0.231125,0,1,16,7,2,6,15
8,0.012683,0.081633,0.045023,1,1,16,7,2,6,15
9,0.039667,0.000000,0.061618,0,1,16,7,2,6,15


In [16]:
x.columns

odds = tf.feature_column.numeric_column('odds')
winners = tf.feature_column.numeric_column('winners')
price = tf.feature_column.numeric_column('price')
hasVideo =  tf.feature_column.categorical_column_with_vocabulary_list('hasVideo',['0','1'])
startDay = tf.feature_column.categorical_column_with_vocabulary_list('startDay',['0','1','2','3','4','5','6'])
startHour = tf.feature_column.categorical_column_with_vocabulary_list('startHour',[str(i) for i in list(range(0,23))])
startDayOfMonth = tf.feature_column.categorical_column_with_vocabulary_list('startDayOfMonth',[str(i) for i in list(range(1,32))])
endDay = tf.feature_column.categorical_column_with_vocabulary_list('endDay',['0','1','2','3','4','5','6'])
endHour = tf.feature_column.categorical_column_with_vocabulary_list('endHour',[str(i) for i in list(range(0,23))])
endDayOfMonth = tf.feature_column.categorical_column_with_vocabulary_list('endDayOfMonth',[str(i) for i in list(range(1,32))])


In [17]:
feat_cols = [odds, winners, price, hasVideo, startDay, startHour, startDayOfMonth, endDay, endHour, endDayOfMonth]

In [18]:
#TRAIN TEST SPLIT

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)

In [21]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs=1000, shuffle=True)

In [22]:
# DNN

In [23]:
emb_hasVideo = tf.feature_column.embedding_column(hasVideo, dimension=2)
emb_startDay = tf.feature_column.embedding_column(startDay, dimension=7)
emb_startHour = tf.feature_column.embedding_column(startHour, dimension=24)
emb_startDayOfMonth = tf.feature_column.embedding_column(startDayOfMonth, dimension=31)
emb_endDay = tf.feature_column.embedding_column(endDay, dimension=7)
emb_endHour = tf.feature_column.embedding_column(endHour, dimension=24)
emb_endDayOfMonth = tf.feature_column.embedding_column(endDayOfMonth, dimension=31)

In [24]:
feat_cols = [odds, winners, price, emb_hasVideo, emb_startDay, emb_startHour, emb_startDayOfMonth, emb_endDay, emb_endHour, emb_endDayOfMonth]

In [25]:
input_func = tf.estimator.inputs.pandas_input_fn(X_train,y_train,batch_size=100,num_epochs=8000,shuffle=True)

In [30]:
dnn_model = tf.estimator.DNNRegressor([10,20,80,160,80,20,10], feature_columns=feat_cols, model_dir='./models/v2')


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './models/v2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020955758BA8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
dnn_model.train(input_fn=input_func,steps=1000000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./models/v2\model.ckpt.
INFO:tensorflow:loss = 19988455000000.0, step = 0
INFO:tensorflow:global_step/sec: 82.4697
INFO:tensorflow:loss = 7489319300000.0, step = 100 (1.212 sec)
INFO:tensorflow:global_step/sec: 119.793
INFO:tensorflow:loss = 6932818000000.0, step = 200 (0.835 sec)
INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 6789758000000.0, step = 300 (0.824 sec)
INFO:tensorflow:global_step/sec: 120.658
INFO:tensorflow:loss = 13228851000000.0, step = 400 (0.828 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 7942607700000.0, step = 500 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 7009188000000.0, step = 600 (0.822 sec)
INFO:tensorflow:gl

INFO:tensorflow:global_step/sec: 119.794
INFO:tensorflow:loss = 6840366700000.0, step = 7600 (0.835 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 7696100600000.0, step = 7700 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 6524517000000.0, step = 7800 (0.824 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 5816315300000.0, step = 7900 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 6769237600000.0, step = 8000 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 6595877500000.0, step = 8100 (0.823 sec)
INFO:tensorflow:global_step/sec: 122.726
INFO:tensorflow:loss = 5679324600000.0, step = 8200 (0.815 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 5599690000000.0, step = 8300 (0.820 sec)
INFO:tensorflow:global_step/sec: 122.726
INFO:tensorflow:loss = 6802288700000.0, step = 8400 (0.815 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflo

INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 5731325600000.0, step = 15400 (0.821 sec)
INFO:tensorflow:global_step/sec: 120.803
INFO:tensorflow:loss = 6396718400000.0, step = 15500 (0.828 sec)
INFO:tensorflow:global_step/sec: 121.242
INFO:tensorflow:loss = 7117758000000.0, step = 15600 (0.825 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 5637660000000.0, step = 15700 (0.824 sec)
INFO:tensorflow:global_step/sec: 120.496
INFO:tensorflow:loss = 6547463700000.0, step = 15800 (0.830 sec)
INFO:tensorflow:global_step/sec: 120.95
INFO:tensorflow:loss = 11221263000000.0, step = 15900 (0.827 sec)
INFO:tensorflow:global_step/sec: 121.684
INFO:tensorflow:loss = 6159249400000.0, step = 16000 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 7655021600000.0, step = 16100 (0.824 sec)
INFO:tensorflow:global_step/sec: 118.379
INFO:tensorflow:loss = 6839349500000.0, step = 16200 (0.845 sec)
INFO:tensorflow:global_step/sec: 118.941
INFO:

INFO:tensorflow:global_step/sec: 117.135
INFO:tensorflow:loss = 4588483600000.0, step = 23200 (0.854 sec)
INFO:tensorflow:global_step/sec: 118.413
INFO:tensorflow:loss = 6963567500000.0, step = 23300 (0.844 sec)
INFO:tensorflow:global_step/sec: 117.342
INFO:tensorflow:loss = 5266831000000.0, step = 23400 (0.853 sec)
INFO:tensorflow:global_step/sec: 117.547
INFO:tensorflow:loss = 6465401700000.0, step = 23500 (0.851 sec)
INFO:tensorflow:global_step/sec: 115.648
INFO:tensorflow:loss = 5331740500000.0, step = 23600 (0.864 sec)
INFO:tensorflow:global_step/sec: 113.553
INFO:tensorflow:loss = 7052123400000.0, step = 23700 (0.881 sec)
INFO:tensorflow:global_step/sec: 116.59
INFO:tensorflow:loss = 6086695300000.0, step = 23800 (0.858 sec)
INFO:tensorflow:global_step/sec: 118.941
INFO:tensorflow:loss = 6297194000000.0, step = 23900 (0.841 sec)
INFO:tensorflow:global_step/sec: 116.862
INFO:tensorflow:loss = 5307983000000.0, step = 24000 (0.857 sec)
INFO:tensorflow:global_step/sec: 118.379
INFO:t

INFO:tensorflow:loss = 6403073000000.0, step = 30900 (0.819 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 9751156000000.0, step = 31000 (0.824 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 5824508300000.0, step = 31100 (0.825 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 5809282000000.0, step = 31200 (0.820 sec)
INFO:tensorflow:global_step/sec: 122.277
INFO:tensorflow:loss = 6364976000000.0, step = 31300 (0.819 sec)
INFO:tensorflow:global_step/sec: 122.277
INFO:tensorflow:loss = 5654623300000.0, step = 31400 (0.818 sec)
INFO:tensorflow:global_step/sec: 118.941
INFO:tensorflow:loss = 14012963000000.0, step = 31500 (0.841 sec)
INFO:tensorflow:global_step/sec: 118.379
INFO:tensorflow:loss = 6170509000000.0, step = 31600 (0.844 sec)
INFO:tensorflow:global_step/sec: 118.778
INFO:tensorflow:loss = 6070516000000.0, step = 31700 (0.842 sec)
INFO:tensorflow:global_step/sec: 108.48
INFO:tensorflow:loss = 6687567000000.0, step =

...

INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 3361242000000.0, step = 961900 (0.823 sec)
INFO:tensorflow:global_step/sec: 122.128
INFO:tensorflow:loss = 4920935700000.0, step = 962000 (0.819 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 4314536300000.0, step = 962100 (0.820 sec)
INFO:tensorflow:global_step/sec: 122.128
INFO:tensorflow:loss = 4635065600000.0, step = 962200 (0.819 sec)
INFO:tensorflow:global_step/sec: 122.277
INFO:tensorflow:loss = 5664614000000.0, step = 962300 (0.818 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 4579066000000.0, step = 962400 (0.822 sec)
INFO:tensorflow:global_step/sec: 122.576
INFO:tensorflow:loss = 4394687700000.0, step = 962500 (0.815 sec)
INFO:tensorflow:global_step/sec: 122.277
INFO:tensorflow:loss = 4990900400000.0, step = 962600 (0.819 sec)
INFO:tensorflow:global_step/sec: 121.242
INFO:tensorflow:loss = 4827896600000.0, step = 962700 (0.824 sec)
INFO:tensorflow:global_step/sec: 121.

INFO:tensorflow:global_step/sec: 122.576
INFO:tensorflow:loss = 5465302000000.0, step = 969600 (0.817 sec)
INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 4883998600000.0, step = 969700 (0.823 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 3936236300000.0, step = 969800 (0.823 sec)
INFO:tensorflow:global_step/sec: 122.277
INFO:tensorflow:loss = 4074381500000.0, step = 969900 (0.817 sec)
INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 3815880300000.0, step = 970000 (0.824 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 3758983700000.0, step = 970100 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 5760020000000.0, step = 970200 (0.823 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 4270328400000.0, step = 970300 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 4717384600000.0, step = 970400 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.

INFO:tensorflow:global_step/sec: 121.389
INFO:tensorflow:loss = 4359505300000.0, step = 977300 (0.824 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 5006385000000.0, step = 977400 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 4977440000000.0, step = 977500 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 4810434000000.0, step = 977600 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 4182666000000.0, step = 977700 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 4463119000000.0, step = 977800 (0.823 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 4943863000000.0, step = 977900 (0.821 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 4655193500000.0, step = 978000 (0.821 sec)
INFO:tensorflow:global_step/sec: 122.726
INFO:tensorflow:loss = 4282347200000.0, step = 978100 (0.815 sec)
INFO:tensorflow:global_step/sec: 121.

INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 4694765000000.0, step = 985000 (0.822 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 4627658400000.0, step = 985100 (0.821 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 5047823400000.0, step = 985200 (0.822 sec)
INFO:tensorflow:global_step/sec: 122.277
INFO:tensorflow:loss = 4322840800000.0, step = 985300 (0.818 sec)
INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 4945222300000.0, step = 985400 (0.823 sec)
INFO:tensorflow:global_step/sec: 121.536
INFO:tensorflow:loss = 4587605000000.0, step = 985500 (0.823 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 3975094200000.0, step = 985600 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.831
INFO:tensorflow:loss = 5361983600000.0, step = 985700 (0.821 sec)
INFO:tensorflow:global_step/sec: 122.277
INFO:tensorflow:loss = 4560556000000.0, step = 985800 (0.818 sec)
INFO:tensorflow:global_step/sec: 122.

INFO:tensorflow:global_step/sec: 122.128
INFO:tensorflow:loss = 4478122600000.0, step = 992700 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 3995813300000.0, step = 992800 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 4680268600000.0, step = 992900 (0.820 sec)
INFO:tensorflow:global_step/sec: 122.726
INFO:tensorflow:loss = 3582636000000.0, step = 993000 (0.815 sec)
INFO:tensorflow:global_step/sec: 123.027
INFO:tensorflow:loss = 5334562000000.0, step = 993100 (0.813 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 5482409400000.0, step = 993200 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.98
INFO:tensorflow:loss = 4441760600000.0, step = 993300 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.979
INFO:tensorflow:loss = 6114881000000.0, step = 993400 (0.820 sec)
INFO:tensorflow:global_step/sec: 121.683
INFO:tensorflow:loss = 4735884000000.0, step = 993500 (0.822 sec)
INFO:tensorflow:global_step/sec: 119.7

In [34]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)


In [35]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-29-12:54:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/v2\model.ckpt-1000000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-29-12:54:48
INFO:tensorflow:Saving dict for global step 1000000: average_loss = 54906745000.0, global_step = 1000000, label/mean = 314629.0, loss = 548862330000.0, prediction/mean = 311355.16
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000000: ./models/v2\model.ckpt-1000000


{'average_loss': 54906745000.0,
 'label/mean': 314629.0,
 'loss': 548862330000.0,
 'prediction/mean': 311355.16,
 'global_step': 1000000}

In [36]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=10, num_epochs=1, shuffle=False)
predictions = dnn_model.predict(pred_input_func)

In [37]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'])



INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./models/v2\model.ckpt-1000000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [38]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,final_preds)**0.5

234321.90443008227

In [ ]:
final_preds
